In [1]:
import numpy as np
import pandas as pd
import torch
import yaml
import torch.nn as nn
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorboardX import SummaryWriter

from lr_scheduler import build_scheduler

import datetime

with open('lr_config.yaml', 'r') as f:
    lr_config = yaml.load(f, yaml.FullLoader) 

# load dataset, split into input (X) and output (y) variables
dataframe = pd.read_csv("ionosphere.csv", header=None)
dataset = dataframe.values
X = dataset[:,0:34].astype(float)
y = dataset[:,34]

# encode class values as integers
encoder = LabelEncoder()
encoder.fit(y)
y = encoder.transform(y)

# convert into PyTorch tensors
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32).reshape(-1, 1)

# train-test split for evaluation of the model
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, shuffle=True)

# create model
model = nn.Sequential(
    nn.Linear(34, 34),
    nn.ReLU(),
    nn.Linear(34, 1),
    nn.Sigmoid()
)
timestamp = datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S")

global_step = 0
train_writer = SummaryWriter(f"test_runs/train/log_{timestamp}")

# Train the model
n_epochs = 90
batch_size = 24
batch_start = torch.arange(0, len(X_train), batch_size)
lr = 0.1
loss_fn = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=lr)
weight_decay = 0.0004
base_lr = 0.1
lr_decay_rate = 0.1
step= [60, 80]
warm_up_epoch = 5

def adjust_learning_rate(epoch, optimizer, base_lr, warm_up_epoch, lr_decay_rate,step):
    # print(f"adjust learning rate, using warm up, epoch: {warm_up_epoch}")
    if epoch < warm_up_epoch:
        lr = base_lr * (epoch + 1) / warm_up_epoch
    else:
        lr = base_lr * ( lr_decay_rate ** np.sum(epoch >= np.array(step)))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
    return lr

def lr_lambda(epoch, optimizer, base_lr, warm_up_epoch, lr_decay_rate,step):
    return adjust_learning_rate(epoch, optimizer, base_lr, warm_up_epoch, lr_decay_rate,step)


# scheduler = build_scheduler(lr_config, optimizer, len(batch_start))
scheduler = lr_scheduler.LambdaLR(optimizer, lr_lambda)
# scheduler = lr_scheduler.LinearLR(optimizer, start_factor=1.0, end_factor=0.5, total_iters=30)


model.train()

for epoch in range(n_epochs):
    train_writer.add_scalar('epoch', epoch, global_step)
    global_step += 1
    train_loss, train_acc = 0, 0
    for start in batch_start:
        X_batch = X_train[start:start+batch_size]
        y_batch = y_train[start:start+batch_size]
        y_pred = model(X_batch)
        loss = loss_fn(y_pred, y_batch)
        train_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_acc += float((y_pred.round() == y_batch).float().mean())
    
    train_loss = train_loss / len(batch_start)
    train_acc = train_acc / len(batch_start)
      
    before_lr = optimizer.param_groups[0]["lr"]
    scheduler.step()
    after_lr = optimizer.param_groups[0]["lr"]
    print("Epoch %d: SGD lr %.4f -> %.4f" % (epoch, before_lr, after_lr))
    train_writer.add_scalar('train_loss', train_loss, global_step)
    train_writer.add_scalar('train_acc', train_acc, global_step)
    train_writer.add_scalar('learning_rate', before_lr, global_step)

# evaluate accuracy after training
model.eval()
y_pred = model(X_test)
acc = (y_pred.round() == y_test).float().mean()
acc = float(acc)
print("Model accuracy: %.2f%%" % (acc*100))
train_writer.close()

Epoch 0: SGD lr 0.0020 -> 0.0040
Epoch 1: SGD lr 0.0040 -> 0.0060
Epoch 2: SGD lr 0.0060 -> 0.0080
Epoch 3: SGD lr 0.0080 -> 0.0100
Epoch 4: SGD lr 0.0100 -> 0.0100
Epoch 5: SGD lr 0.0100 -> 0.0100
Epoch 6: SGD lr 0.0100 -> 0.0100
Epoch 7: SGD lr 0.0100 -> 0.0100
Epoch 8: SGD lr 0.0100 -> 0.0100
Epoch 9: SGD lr 0.0100 -> 0.0100
Epoch 10: SGD lr 0.0100 -> 0.0100
Epoch 11: SGD lr 0.0100 -> 0.0100
Epoch 12: SGD lr 0.0100 -> 0.0100
Epoch 13: SGD lr 0.0100 -> 0.0100
Epoch 14: SGD lr 0.0100 -> 0.0100
Epoch 15: SGD lr 0.0100 -> 0.0100
Epoch 16: SGD lr 0.0100 -> 0.0100
Epoch 17: SGD lr 0.0100 -> 0.0100
Epoch 18: SGD lr 0.0100 -> 0.0100
Epoch 19: SGD lr 0.0100 -> 0.0100
Epoch 20: SGD lr 0.0100 -> 0.0100
Epoch 21: SGD lr 0.0100 -> 0.0100
Epoch 22: SGD lr 0.0100 -> 0.0100
Epoch 23: SGD lr 0.0100 -> 0.0100
Epoch 24: SGD lr 0.0100 -> 0.0100
Epoch 25: SGD lr 0.0100 -> 0.0100
Epoch 26: SGD lr 0.0100 -> 0.0100
Epoch 27: SGD lr 0.0100 -> 0.0100
Epoch 28: SGD lr 0.0100 -> 0.0100
Epoch 29: SGD lr 0.0100 